In [1]:
import numpy as np
import cv2
import argparse
from collections import deque

In [2]:
cap=cv2.VideoCapture(0)
#VideoCapture:Class for video capturing from video files, image sequences or cameras.
#cv2.VideoCapture(device):id of the opened video capturing device (i.e. a camera index). If there is a single camera connected, just pass 0.

In [3]:
pts = deque(maxlen=64)

In [4]:
Lower_green = np.array([110,50,50])
Upper_green = np.array([130,255,255])

In [5]:
while True:
    ret, img=cap.read() #.read():Grabs, decodes and returns the next video frame..
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV) #.cvtColor():Converts BGR image into HSV image.
    kernel=np.ones((5,5),np.uint8)
    mask=cv2.inRange(hsv,Lower_green,Upper_green) #Threshold the HSV image to get only blue colors.
    mask = cv2.erode(mask, kernel, iterations=2) #It erodes away the boundaries of foreground object.
    mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel) #It is useful in removing noise.
    mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel) #It is useful in closing small holes inside the foreground objects.
    mask = cv2.dilate(mask, kernel, iterations=1) #It increases the white region in the image or size of foreground object increases.
    #(erosion removes white noises, but it also shrinks our object. So we dilate it.)
    res=cv2.bitwise_and(img,img,mask=mask) #To merge two images. 
    cnts,heir=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:] #Used to find the contours.(first one is source image, second is contour retrieval mode, third is contour approximation method).
    center = None
    
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c) #to get minimum enclosing circle around an object of an image.
        M = cv2.moments(c) #to extract centroid.
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])) #Centroid is given by this relations.
        if radius > 5:
            cv2.circle(img, (int(x), int(y)), int(radius),(0, 255, 255), 2) #to draw circle with the given dimensions with thickness 2.
            cv2.circle(img, center, 5, (0, 0, 255), -1) #to draw a filled circle(negative thickness) with given dimensions.
            
    pts.appendleft(center) #used to append all the centroids of the blobs detected to left side of the list 'pts'.
    for i in range(1,len(pts)): 
        if pts[i-1] is None or pts[i] is None:
               continue
        
        thick=int(np.sqrt(len(pts)/(i+1))*2.5) #to find thickness of the line to be drawn.
        cv2.line(img, pts[i-1],pts[i],(0,0,225),thick) #to draw the line
    
    cv2.imshow("Frame", img)
    cv2.imshow("mask",mask)
    cv2.imshow("res",res)
    
    if cv2.waitKey(60) & 0xFF == ord('q'):
        #to cleanup the camera and close any open windows
        cap.release()
        cv2.destroyAllWindows()
        break